In [148]:
import pandas as pd
import numpy as np
from pymongo import MongoClient
from bs4 import BeautifulSoup
import pprint
# Requests sends and recieves HTTP requests.
import requests
import time

In [84]:
%load_ext autoreload
%autoreload 2

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
options = Options()
options.headless = True
from selenium.common.exceptions import WebDriverException

In [59]:
from data_collect import frag_url_scrape
from cologne_scrape import frag_info_scrape

In [5]:
male_frag=['https://www.fragrantica.com/search/?spol=male&osobine.EN=Aromatic',
           'https://www.fragrantica.com/search/?spol=male&osobine.EN=Aromatic%20Aquatic',
           'https://www.fragrantica.com/search/?spol=male&osobine.EN=Aromatic%20Fougere',
           'https://www.fragrantica.com/search/?spol=male&osobine.EN=Aromatic%20Fruity',
           'https://www.fragrantica.com/search/?spol=male&osobine.EN=Aromatic%20Green',
           'https://www.fragrantica.com/search/?spol=male&osobine.EN=Aromatic%20Spicy',
           'https://www.fragrantica.com/search/?spol=male&osobine.EN=Chypre',
           'https://www.fragrantica.com/search/?spol=male&osobine.EN=Chypre%20Floral',
           'https://www.fragrantica.com/search/?spol=male&osobine.EN=Chypre%20Fruity',
           'https://www.fragrantica.com/search/?spol=male&osobine.EN=Citrus',
           'https://www.fragrantica.com/search/?spol=male&osobine.EN=Citrus%20Aromatic',
           'https://www.fragrantica.com/search/?spol=male&osobine.EN=Citrus%20Gourmand',
           'https://www.fragrantica.com/search/?spol=male&osobine.EN=Floral',
           'https://www.fragrantica.com/search/?spol=male&osobine.EN=Floral%20Aldehyde',
           'https://www.fragrantica.com/search/?spol=male&osobine.EN=Floral%20Aquatic',
           'https://www.fragrantica.com/search/?spol=male&osobine.EN=Floral%20Fruity',
           'https://www.fragrantica.com/search/?spol=male&osobine.EN=Floral%20Fruity%20Gourmand',
           'https://www.fragrantica.com/search/?spol=male&osobine.EN=Floral%20Green',
           'https://www.fragrantica.com/search/?spol=male&osobine.EN=Floral%20Woody%20Musk',
           'https://www.fragrantica.com/search/?spol=male&osobine.EN=Leather',
           'https://www.fragrantica.com/search/?spol=male&osobine.EN=Oriental',
           'https://www.fragrantica.com/search/?spol=male&osobine.EN=Oriental%20Floral',
           'https://www.fragrantica.com/search/?spol=male&osobine.EN=Oriental%20Fougere',
           'https://www.fragrantica.com/search/?spol=male&osobine.EN=Oriental%20Spicy',
           'https://www.fragrantica.com/search/?spol=male&osobine.EN=Oriental%20Vanilla',
           'https://www.fragrantica.com/search/?spol=male&osobine.EN=Oriental%20Woody',
           'https://www.fragrantica.com/search/?spol=male&osobine.EN=Woody',
           'https://www.fragrantica.com/search/?spol=male&osobine.EN=Woody%20Aquatic',
           'https://www.fragrantica.com/search/?spol=male&osobine.EN=Woody%20Chypre',
           'https://www.fragrantica.com/search/?spol=male&osobine.EN=Woody%20Floral%20Musk',
           'https://www.fragrantica.com/search/?spol=male&main_activity=Fragrances~Cosmetics&osobine.EN=Woody%20Aromatic',
           'https://www.fragrantica.com/search/?spol=male&main_activity=Fashion~Accessories~Natural%20Perfumery~Retailer~Cars~Jewelry~Celebrity~Sport%20Fashion&osobine.EN=Woody%20Aromatic',
          'https://www.fragrantica.com/search/?spol=male&osobine.EN=Woody%20Spicy']

In [6]:
driver = webdriver.Chrome(chrome_options=options)
driver.get('https://www.fragrantica.com/search/?spol=male')
butt=driver.find_element_by_xpath("/html/body/div[1]/div[2]/div[3]/div[1]/div[1]/div/div/div/div[2]/div[1]/div/div[3]/div/div/div/div/div/button")

WebDriverException()
for i in range(1000):
    try:
        WebDriverException()
        butt.click()

    except:
        continue

urls=[]
elements = driver.find_elements_by_css_selector("div.ais-InfiniteHits a")
for elem in elements:
    urls.append(elem.get_attribute("href"))

driver.quit()


In [49]:
np.array(male_urls)

numpy.ndarray

In [42]:
len(male_frag)

33

In [8]:
male_urls=[]

In [41]:
for link in male_frag[33]:
    url=frag_url_scrape(link)
    male_urls.extend(url)
    print('*')

IndexError: list index out of range

In [51]:
pd.DataFrame(np.array(male_urls)).to_csv('data/men_urls.csv')

In [71]:
client = MongoClient('localhost', 27017)

In [73]:
db = client['cologne']
mens_cologne = db['mens_cologne']

In [74]:
mens_cologne.remove({
    'name': 'RainbowDash', 
    'age': 16, 
    'friends': ['TwilightSparkle', 'Applejack', 'Rarity']})

In [13]:
colection = db['cologne']

In [53]:
df=pd.read_csv('data/men_urls.csv')

In [87]:
test=df['0']

In [88]:
test

0       https://www.fragrantica.com/perfume/Caron/Pour...
1       https://www.fragrantica.com/perfume/Calvin-Kle...
2       https://www.fragrantica.com/perfume/Hugo-Boss/...
3       https://www.fragrantica.com/perfume/Amouage/Su...
4       https://www.fragrantica.com/perfume/Lanvin/L-H...
                              ...                        
9157    https://www.fragrantica.com/perfume/Thera-Cosm...
9158    https://www.fragrantica.com/perfume/s-Oliver/T...
9159    https://www.fragrantica.com/perfume/Jequiti/El...
9160    https://www.fragrantica.com/perfume/Estiara/Pr...
9161    https://www.fragrantica.com/perfume/Linn-Young...
Name: 0, Length: 9162, dtype: object

In [86]:
frag_info_scrape(test)

200


NameError: name 'mens_cologne' is not defined

In [115]:
def test_scrape(url):
    '''
    INPUT(String) URL of fragrence review page for a specific fragrence
    '''

    
    r=requests.get(url, headers = {'User-agent': 
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.138 Safari/537.36'},timeout=1000000)
    soup = BeautifulSoup(r.content, "html.parser")
    
    #test for 200 status
    print(r.status_code)
    
    #name of frag
    name=soup.find("div",{"id": "col1"})
    name=name.find_all("span",{'itemprop':'name'})
    name=name[1].text
        
    #user ratings of the frag by demographic/time of day/season
    ratings = soup.find("div", {"id": "diagramresult"})
    rating=ratings.attrs['title']
    rating=rating.split(':')
    
    #dic of ratings - keys:type of rating - values:number of votes
    frag_rating={}
    for idx,data in enumerate(rating[:-1]):
        if idx % 2 == 0:
            frag_rating[data]=float(rating[idx+1])
        else:
            continue
    
    #unique accord dictionaries - Key()
    accords=soup.find("div",{"id": "prettyPhotoGallery"})
    accords=accords.find_all('div')
    accord_dic={}
    for i in [2,5,8,11,14,17]:
        try:
            key=accords[i].find('span').text
            value=accords[i].find('div').attrs['style']
            value=value.split(';')[0].split(': ')[1].split('p')[0]
            accord_dic[key]=float(value)
        except:
            pass
    
    mainpicbox=soup.find("div",{"id": "mainpicbox"})
    
    #image of frag
    try:
        image=mainpicbox.find('img')
        image=image.attrs['src']
    except:
        image=np.nan
    
    
    
    #have it, want it, had it, signature fragrence
    try:
        have_had_want=mainpicbox.find_all('p')[2].text
        have_had_want=have_had_want.split('  ')
    except:
        have_had_want=mainpicbox.find_all('p')[1].text
        have_had_want=have_had_want.split('  ')
    else:
        pass
    try:
        have_it=int(have_had_want[0][-4:])
    except:
        have_it=0
    try:
        had_it=int(have_had_want[1][-4:])
    except:
        had_it=0
    try:
        want_it=int(have_had_want[2][-4:])
    except:
        want_it=0
    try:
        signature=int(have_had_want[3][-4:])
    except:
        signature=0
    
    #price of frag
    prices=[]
    try:
        price=soup.find("div",{"id": "newsocial"})
        price=price.find_all('b')
        for p in price:
            prices.append(p.text)
    except:
        pass
    
    #description of frag
    try:
        description=soup.find('div',{'itemprop':'description'}).text
    except:
        description=''
    
    #col2 scrapes column for sillage/longevity
    col=soup.find('div',{'id':'col1'})
    col1=col.find('div',{'class':'effect6'})
    pyramid=col1.find_all('p')

    
    try:
        user_pyramid=col1.find('div',{'id':'userMainNotes'})
        user_pyramid=user_pyramid.attrs['title']
    except:
        user_pyramid=np.nan
        
    col2=col.find('div',{'class':'longSilBox effect6'})
    col2=col2.find_all('table')
    
    #notes of frag by product description
    top_notes=[]
    top_notes_id=[]
    mid_notes=[]
    mid_notes_id=[]
    base_notes=[]
    base_notes_id=[]

    for i in range(len(pyramid)):
        for j in range(len(pyramid[i].find_all('span',{'class':'rtgNote'}))):
            try:
                if i ==0:
                    top_notes.append(pyramid[i].find_all('span',{'class':'rtgNote'})[j].find('img').attrs['title'])
                    top_notes_id.append(pyramid[i].find_all('span',{'class':'rtgNote'})[j].attrs['title'])
            except:
                continue
            try:
                if i ==1:
                    mid_notes.append(pyramid[i].find_all('span',{'class':'rtgNote'})[j].find('img').attrs['title'])
                    mid_notes_id.append(pyramid[i].find_all('span',{'class':'rtgNote'})[j].attrs['title'])
            except:
                continue
            try:
                if i==2:
                    base_notes.append(pyramid[i].find_all('span',{'class':'rtgNote'})[j].find('img').attrs['title'])
                    base_notes_id.append(pyramid[i].find_all('span',{'class':'rtgNote'})[j].attrs['title'])
            except:
                continue
                
    #user voted strength of notes
    try:
        user_voted_pyramid=user_pyramid.split(';')
    except:
        user_voted_pyramid=np.nan
    
    #sillage
    sillage=col2[1].text.split()[2:]
    close_to_skin=sillage[1]
    radiates_arm_length=sillage[3]
    radiates_6ft=sillage[5]
    fills_room=sillage[7]
    
    #Longevity
    longevity=col2[0].text.split()[2:]
    longevity_30m_1hr=longevity[1]
    longevity_1hr_2hr=longevity[3]
    longevity_3hr_6hr=longevity[5]
    longevity_7hr_12hr=longevity[8]
    longevity_above12hr=longevity[12]
    
    #similiar fragrences voted by the public
    voted_similar_frag=[]
    try:
        reminds_me=col.find('div',{'class':'votes'})
        #reminds_me.find('div',{'class':'fl'})
        remind=reminds_me.find_all('img')
        remind_value=reminds_me.find_all('span')

        for i in range(len(remind)):
            voted_similar_frag.append(remind[i].attrs['title'])
            voted_similar_frag.append(remind_value[i].text)
    except:
        pass
    
    #user reviews
    
    user_reviews={}
    user_review_date=[]
    try:
        user=col.find_all('div',{'class':'pwq'})
        for i in range(len(user)):
            user_name=user[i].find('div',{'class':'avatar'}).find('span').text.split('\n')[0]
            if '.' in user_name:
                user_name=user_name.replace('.',' ')
            user_review=user[i].find('div',{'class':'revND'}).text.replace('\n','')
            user_reviews[user_name]=user_review
            user_review_date.append(user[i].find('div',{'class':'revND'}).text.split('\n')[1][1:])
    except:
        pass

        
    mens_cologne.insert_one({
    'name': name, 
    'frag rating': frag_rating, 
    'main accords': accord_dic,
    'image':image,
    'have it':have_it,
    'had it':had_it,
    'want it':want_it,
    'signature':signature,
    'price':prices,
    'description':description,
    'top notes':top_notes,
    'top notes id':top_notes_id,
    'mid notes':mid_notes,
    'mid notes id':mid_notes_id,
    'base notes':base_notes,
    'base notes id':base_notes_id,
    'user voted notes': user_voted_pyramid,
    'close to skin':close_to_skin,
    'radiates about arm length':radiates_arm_length,
    'radiates 6ft':radiates_6ft,
    'fills room':fills_room,
    '30min to 1hr':longevity_30m_1hr,
    '1hr to 2hr':longevity_1hr_2hr,
    '3hr to 6hr':longevity_3hr_6hr,
    '7hr to 12hr':longevity_7hr_12hr,
    'greater than 12hr':longevity_above12hr,
    'similiar fragrences by user vote':voted_similar_frag,
    'user reviews':user_reviews,
    'user review date':user_review_date})
    time.sleep(5)

In [117]:
for page in test[99:]:
    test_scrape(page)

200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
429


AttributeError: 'NoneType' object has no attribute 'find_all'

In [91]:
test[1]

'https://www.fragrantica.com/perfume/Calvin-Klein/Reveal-Men-27996.html'

In [92]:
for i in range(1000)

1    https://www.fragrantica.com/perfume/Calvin-Kle...
2    https://www.fragrantica.com/perfume/Hugo-Boss/...
Name: 0, dtype: object

In [118]:
mens_cologne.count_documents({})

123

In [110]:
test[77]

'https://www.fragrantica.com/perfume/Jean-Paul-Gaultier/Le-Male-Summer-2013-17823.html'

In [120]:
df=pd.DataFrame(list(mens_cologne.find()))

In [122]:
df=df.drop(columns=['age','friends'])

In [126]:
df=df.drop(0)

In [133]:
df=df.drop(columns='_id')

In [136]:
pd.set_option('max_columns', None)

In [146]:
df['have it']

1      1410.0
2      1410.0
3       636.0
4       556.0
5         0.0
        ...  
118       0.0
119       0.0
120       0.0
121       0.0
122       0.0
Name: have it, Length: 122, dtype: float64

In [147]:
test[120]

'https://www.fragrantica.com/perfume/Adidas/Adidas-Sport-Fever-7209.html'